In [138]:
import os
from binance.client import Client
import pandas as pd
import time
import requests  
from datetime import datetime, timedelta

def get_historic_data(days):

    # Calculate the timestamps for the binance api function
    until = datetime.now()
    since = until - timedelta(days = days)
    client = Client()
    candle = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, str(since), str(until))

    data = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    data['dateTime'] = pd.to_datetime(data['dateTime'], unit='ms')
    data = data.drop(['quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)

    return (data)


# binance_data = get_historic_data(2)
# binance_data['id'] = range(0,len(binance_data))
# binance_data = binance_data.set_index('id')
# print (binance_data)

In [139]:
api_key = '2WXI6JX92VIJ02ZMIO9PudPVv3S'

def get_glassnode_data(days): 
    asset = "BTC" 
    no_of_datapoints = days*24 
    time_interval = '1h' 
    since = int(time.time() - no_of_datapoints*60*60) 
    params = pd.read_csv('params.csv')['path']

#     print (datetime.fromtimestamp(int(time.time())))
    data = pd.DataFrame()

    for n, param in enumerate(params):

        temp = pd.DataFrame()
        res = requests.get('https://api.glassnode.com' + param,
            params={'a': asset, 'i': time_interval, 's': since, 'api_key': api_key})

        param_name = param.split('/')[-1]

        if n == 0:
            data = pd.read_json(res.text)
        else:
            temp = pd.read_json(res.text)
            data = pd.merge_asof(data, temp, on='t')
        data = data.rename(columns={"v": param_name})
        time.sleep(1)

    return (data)


In [140]:
def get_current_price():
    url = 'https://api.binance.com'
    path = '/api/v3/ticker/price'

    response = requests.get(url + path, params={'symbol':'BTCUSDT'})
    price = response.json()
    return (price)


In [ ]:
while (True):
    if (int(time.time() % 3600)) < 4:
#     if True:
        time.sleep(3)
        price = get_current_price()
        
        print ('datetime:           {}'.format(datetime.now()))
        print ('timestamp:          {}'.format(int(time.time())))
        print ('current price:      {}'.format(int(float(price['price']))))
      
        binance_data = get_historic_data(2)
        binance_data['id'] = range(0, len(binance_data))
        binance_data = binance_data.set_index('id')

        glassnode_data = get_glassnode_data(2)
        glassnode_data['id'] = range(0, len(glassnode_data))
        glassnode_data = glassnode_data.set_index('id')
        
        data = pd.merge_asof(glassnode_data, binance_data, right_on='id', left_on='id')
        
        price = get_current_price()
        
        print ('binance close:      {}'.format(int(float(data.iloc[-1]['close']))))
        print ('binance open:       {}'.format(int(float(data.iloc[-1]['open']))))
        print ('price usd close:    {}'.format(int(float(data.iloc[-1]['price_usd_close']))))
        print ('current price:      {}'.format(int(float(price['price']))))
        print ('close time:         {}'.format(int(float(data.iloc[-1]['closeTime'])/1000)))
        print ('glassnode t:        {}'.format(int(float(data.iloc[-1]['t']))))
        print ('current t:          {}'.format(int(time.time())))
        
        print ('-----------')
    else:
        print ((3600 - int(time.time()) % 3600), end = "\r")
        time.sleep(3)

datetime:           2024-07-27 03:00:04.801977
timestamp:          1722070804
current price:      67976
binance close:      68028
binance open:       67657
price usd close:    68139
current price:      67962
close time:         1722049199
glassnode t:        1722063600
current t:          1722070871
-----------
datetime:           2024-07-27 04:00:06.254734
timestamp:          1722074406
current price:      68219
binance close:      67798
binance open:       68028
price usd close:    67949
current price:      68220
close time:         1722052799
glassnode t:        1722067200
current t:          1722074470
-----------
datetime:           2024-07-27 05:00:05.282296
timestamp:          1722078005
current price:      68155
binance close:      67866
binance open:       67797
price usd close:    68214
current price:      68155
close time:         1722056399
glassnode t:        1722070800
current t:          1722078067
-----------
datetime:           2024-07-27 06:00:05.787329
timestamp:    

binance close:      67338
binance open:       67940
price usd close:    67449
current price:      67576
close time:         1722142799
glassnode t:        1722157200
current t:          1722164470
-----------
datetime:           2024-07-28 06:00:06.173727
timestamp:          1722168006
current price:      67864
binance close:      67422
binance open:       67338
price usd close:    67580
current price:      67798
close time:         1722146399
glassnode t:        1722160800
current t:          1722168070
-----------
datetime:           2024-07-28 07:00:05.404597
timestamp:          1722171605
current price:      68000
binance close:      67486
binance open:       67422
price usd close:    67881
current price:      68008
close time:         1722149999
glassnode t:        1722164400
current t:          1722171669
-----------
datetime:           2024-07-28 08:00:04.377326
timestamp:          1722175204
current price:      68005
binance close:      67560
binance open:       67486
price usd

datetime:           2024-07-29 08:00:05.339309
timestamp:          1722261605
current price:      69273
binance close:      69528
binance open:       69588
price usd close:    69769
current price:      69285
close time:         1722239999
glassnode t:        1722254400
current t:          1722261667
-----------
datetime:           2024-07-29 09:00:05.433125
timestamp:          1722265205
current price:      68208
binance close:      69457
binance open:       69528
price usd close:    69222
current price:      68159
close time:         1722243599
glassnode t:        1722258000
current t:          1722265271
-----------
datetime:           2024-07-29 10:00:05.995312
timestamp:          1722268805
current price:      68097
binance close:      69520
binance open:       69457
price usd close:    68321
current price:      68000
close time:         1722247199
glassnode t:        1722261600
current t:          1722268868
-----------
datetime:           2024-07-29 11:00:06.209760
timestamp:    

In [ ]:
data.to_csv('prediction_data.csv')